# Joe's code place

![Hackerman](https://i.kym-cdn.com/entries/icons/mobile/000/021/807/ig9OoyenpxqdCQyABmOQBZDI0duHk2QZZmWg2Hxd4ro.jpg)

# Code

In [1]:
# Imports
import os
import pandas as pd
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
""" Ticker Library init
portfolio = {'Sector':[],'Company Name':[],'Ticker':[]}
hello = [
    ['Technology','Netflix','NFLX'],
    ['Technology','Meta','FB'],
    ['Technology','Uber','UBER'],
    ['Technology','Microchip Technology','MCHP'],
    ['Technology','AirBnB','ABNB'],
    ['Energy','Diamondback Energy Inc.','FANG'],
    ['Energy','Marathon Oil Corp.','MRO'],
    ['Energy','Devon Energy Corp.','DVN'],
    ['Energy','SunPower Corp.','SPWR'],
    ['Energy','Renewable Energy Group Inc.','REGI'],
    ['Finance','McKinsey & Company','MTRX'],
    ['Finance','BlackRock','BLK'],
    ['Finance','PayPal','PYPL'],
    ['Finance','Mercadolibre Inc','MELI'],
    ['Finance','SoFi','SOFI']
]
for row in hello:
    portfolio['Sector'].append(row[0])
    portfolio['Company Name'].append(row[1])
    portfolio['Ticker'].append(row[2])
portfolio = pd.DataFrame(portfolio).set_index('Sector')
portfolio.to_csv('./Data/Cleaned_Data/Ticker_library.csv')
"""

" Ticker Library init\nportfolio = {'Sector':[],'Company Name':[],'Ticker':[]}\nhello = [\n    ['Technology','Netflix','NFLX'],\n    ['Technology','Meta','FB'],\n    ['Technology','Uber','UBER'],\n    ['Technology','Microchip Technology','MCHP'],\n    ['Technology','AirBnB','ABNB'],\n    ['Energy','Diamondback Energy Inc.','FANG'],\n    ['Energy','Marathon Oil Corp.','MRO'],\n    ['Energy','Devon Energy Corp.','DVN'],\n    ['Energy','SunPower Corp.','SPWR'],\n    ['Energy','Renewable Energy Group Inc.','REGI'],\n    ['Finance','McKinsey & Company','MTRX'],\n    ['Finance','BlackRock','BLK'],\n    ['Finance','PayPal','PYPL'],\n    ['Finance','Mercadolibre Inc','MELI'],\n    ['Finance','SoFi','SOFI']\n]\nfor row in hello:\n    portfolio['Sector'].append(row[0])\n    portfolio['Company Name'].append(row[1])\n    portfolio['Ticker'].append(row[2])\nportfolio = pd.DataFrame(portfolio).set_index('Sector')\nportfolio.to_csv('./Data/Cleaned_Data/Ticker_library.csv')\n"

## Twitter

In [3]:
import tweepy

In [4]:
def init_twitter_API():
    return tweepy.API(
        tweepy.OAuth1UserHandler(
            # Generated when you setup your Twitter Dev account
            os.getenv("twitter_api_consumer_key"),
            os.getenv("twitter_api_consumer_secret_key"),
            # You have to generate the below after you setup your Twitter Dev Account
            os.getenv("twitter_api_access_key"),
            os.getenv("twitter_api_access_secret_key")
        )
    )

# tw = init_twitter_API()

## Python Edgar

In [5]:
from secedgar import CompanyFilings, FilingType

def sec_filings(sec_filings, your_name, your_email, start, end, filing_type:FilingType=FilingType.FILING_10Q):
    return CompanyFilings(cik_lookup=sec_filings,
                                filing_type=filing_type,
                                start_date=start,
                                end_date=end,
                                user_agent=f'{your_name} ({your_email})')

In [6]:
tickers = pd.read_csv('./Data/Cleaned_Data/Ticker_library.csv')['Ticker'].to_list()

In [7]:
from sec_edgar_downloader import Downloader

In [8]:
def create_SEC(path,tickers,filing_type,after_date,before_date=None):
    dl = Downloader(f'{path}')
    for ticker in tickers:
        dl.get(filing_type, ticker, after=after_date,before=before_date)

In [9]:
# create_SEC('./Data/Raw_Data/',tickers,"10-Q",after_date='2012-06-01')

In [10]:
# df = pd.read_html('.\Data\Raw_Data\sec-edgar-filings\ABNB/10-Q/0001559720-21-000017/full-submission.txt')

In [11]:
def txt_finder():
    txts = []
    for root, dirs, files in os.walk(".", topdown=False):
        for name in files:
            os.path.join(root, name)
            # print(name)
            if name[-4:]=='.txt':
                txts.append(f'{root}/{name}')
        for name in dirs:
            os.path.join(root, name)
    return txts
sec_paths = txt_finder()

In [12]:
sec_paths[0]

'.\\Data\\Raw_Data\\sec-edgar-filings\\ABNB\\10-Q\\0001559720-21-000017/full-submission.txt'

In [14]:
def path_part(path,before,after):
    beforeNum = path.find(before)+len(before)
    afterNum = path.find(after)
    return path[beforeNum:afterNum]

def sec_df_from_paths(sec_paths,filing_type):
    data = {
        'Ticker':[],
        'DocName':[],
        'DocText':[]
    }
    for sec in sec_paths:
        data['Ticker'].append(path_part(sec,'sec-edgar-filings\\',f'\\{filing_type}'))
        data['DocName'].append(path_part(sec,f'\\{filing_type}\\','/full-submission'))
        
        sec_txt = open(sec, "r")
        data['DocText'] = sec_txt.read()
        sec_txt.close()
    return pd.DataFrame(data)

df_10Q = sec_df_from_paths(sec_paths,'10-Q')

In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(df_10Q['DocText'].loc[0],'html.parser')

In [ ]:
soup.get_text()

In [ ]:
import re
from bs4 import BeautifulSoup
def html_clean(html_txt,loops=2): # This loops to make sure <*<*>*> and such are caught. (TODO might not work might catch <*<*> and miss the *>)
    for loop in range(loops):
        # removes <*>, {*}, and [*] from the HTML
        html_txt = re.sub("[\<\{\[].*?[\>\}\]]"," ",html_txt)
    return html_txt
def soup_clean(html_txt):
    return BeautifulSoup(html_txt, 'html.parser').get_text()
df_10Q_Test = df_10Q.copy()
# df_10Q_Test['DocText'] = [html_clean(txt) for txt in df_10Q_Test['DocText'].to_list()]
df_10Q_Test['DocText'] = [soup_clean(txt) for txt in df_10Q_Test['DocText'].to_list()]
df_10Q_Test

In [ ]:
df_10Q_Test.to_csv('.\Data\Cleaned_Data\sec_df.csv')

## PRAW reddit API